In [277]:
using ITensors, ITensorMPS
using LinearAlgebra
using Plots

#sites = siteinds(psi)

N = 4
s = siteinds("Qubit",N)


4-element Vector{Index{Int64}}:
 (dim=2|id=872|"Qubit,Site,n=1")
 (dim=2|id=643|"Qubit,Site,n=2")
 (dim=2|id=565|"Qubit,Site,n=3")
 (dim=2|id=546|"Qubit,Site,n=4")

In [278]:
gates = ITensor[]

function H_gate(x)
    Hadamard = op("H",s[x]) # op  is  a function which converts matrices to ITensors
    
end

function swap_gate(i,j)
    swap = [ 1 0 0 0 ; 0 0 1 0; 0 1 0 0; 0 0 0 1] 
    swap_gate = op(swap, s[i] , s[j])
    
end

function phase_gate(x,i,j)
    phase = [ 1 0 0 0 ; 0 1 0 0; 0 0 1 0; 0 0 0 exp(im * 2*π/2^x)] 
    phase_gate = op(phase, s[i] , s[j])
    # s[i] and s[j] are the control qubit 
    
end

function phase_gate1(i,j,θ)
    phase = [ 1 0 0 0 ; 0 1 0 0; 0 0 1 0; 0 0 0 exp(im * θ)] 
    phase_gate = op(phase, s[i] , s[j])
    # s[i] and s[j] are the control qubit 
    
end

phase_gate1 (generic function with 1 method)

In [266]:

function generate_basis_states(n)
    basis_states = []
    for i in 0:(2^n - 1)
        binary_str = bitstring(i)[end-n+1:end]  # Get last n bits of bitstring
        push!(basis_states, collect(binary_str))  # Collect splits string into array of characters
    end
    return basis_states
end

function generate_basis_states1(n)
    basis_states = []
    for i in 0:(2^n - 1)
        binary_str = bitstring(i)[end-n+1:end]  # Get last n bits of bitstring
        push!(basis_states, collect(reverse(binary_str)))  # Collect splits string into array of characters
    end
    return basis_states
end


MPS_states = []
basis_states = generate_basis_states(N)

for i in 1:(2^N)
    push!(MPS_states, MPS(s,string.(basis_states[i])))
end

MPS_states1 = []
basis_states1 = generate_basis_states1(N)

for i in 1:(2^N)
    push!(MPS_states1, MPS(s,string.(basis_states1[i])))
end

using HDF5
f = 0 
f1 = 0

for (i, mps) in enumerate(MPS_states)
    f = h5open("Basis_states/MPS_$N/MPS_create_$i.h5","w")
    write(f,"M", mps)
end
close(f)

for (i, mps) in enumerate(MPS_states1)
    f1 = h5open("Basis_states/MPS_$N/MPS_measure_$i.h5","w")
    write(f1,"M1", mps)
end
close(f1)

In [267]:
# Taking the input 

x = range(0, stop=2π, length=2^N)
input_array = cos.(x)
    
#ψ = MPS(input_array,s;cutoff=cutoff,maxdim=maxdim)
using HDF5
MPS1 = []
for i in 1:2^N
    f1 = h5open("Basis_states/MPS_$N/MPS_create_$i.h5","r")
    mps1 = read(f1,"M",MPS)
    push!(MPS1,mps1)
    close(f1)
end
array = input_array / norm(input_array) # Input

ψ = array[1] *MPS1[1]
for i in 2:2^N
    
    ψ += array[i] * MPS1[i]
end

println(maxlinkdim(ψ))

5


In [268]:
# Define the QFT MPO
function qft_mpo(N::Int, sites)
    # Initialize MPO with sites
    W = MPO(sites,"I")  
    
    cutoff=1e-15
    # Apply Hadamard and phase gates to build the QFT MPO
    
    
    for i in 1:N
        W = apply(op("H",sites[i]),W,cutoff=cutoff)

        for j in i+1:N
            θ = π / 2^(j-i)
            P = phase_gate1(i,j,θ)
            # Update MPO by combining with phase gate tensor
            W = apply(P,W,cutoff=cutoff)


        end
            
    end
    
    return W
end

W = qft_mpo(N, s)
println(maxlinkdim(W))
orthogonalize!(W, 1)   # Orthogonalize W at the first index
orthogonalize!(ψ, 1) # Orthogonalize Psi at the first index

f3 = h5open("Basis_states/MPO_$N.h5","w")
write(f3,"W", W)
close(f3)

f4 = h5open("Basis_states/MPO_$N.h5","r")
W1 = read(f4,"W",MPO)
close(f4)

start = time()
# Contract MPO and MPS
result = contract(W1, ψ)
end_time = time()

9


1.730238093385203e9

In [269]:
inner_products = []

MPS2 = []
for i in 1:2^N
    f2 = h5open("Basis_states/MPS_$N/MPS_measure_$i.h5","r")
    mps1 = read(f2,"M1",MPS)
    close(f2)
    push!(MPS2,mps1)
    
end

for i in 1:(2^N)
    push!(inner_products, inner(MPS2[i], result))
    print(inner(MPS2[i], result)*norm(input_array)*2^(N/2))
    println()
end



0.999650005850407 - 2.1421342741332037e-8im
131072.24998290226 - 1.5707633382955513im
-0.33319994093835986 + 0.0010104129909301377im
-0.12501022381736104 + 3.2656102811091042e-6im
-0.06621885421395073 + 0.0008231875551677188im
-0.04166452028035961 - 9.636283766506068e-5im
-0.03674471092311847 - 0.0026176655844963996im
-0.020614583349519995 - 7.67158173399383e-5im
-9.102872502825111e-8 + 3.061543256958173e-9im
-0.012041437993024373 + 0.00042218582077620427im
3.10625243855806e-8 - 1.2153641437571086e-9im
1.2862032099612674e-8 - 4.820424199093056e-10im
8.38753141387097e-9 - 3.816347186633405e-10im
7.961599048789142e-9 - 3.53327725141061e-10im
1.4403685099265429e-8 - 4.5247261657039325e-10im
-0.004334064667717786 + 0.00023914285530216633im
-2.76200780226433e-8 + 2.8028072386965895e-9im
-0.003621488871463018 + 0.0003675421954212339im
9.203397499419588e-9 - 9.62139441307483e-10im
3.4539918273436774e-9 - 3.5059240071732727e-10im
1.8273021716752246e-9 - 2.0840811195350204e-10im
1.1514502835735

Excessive output truncated after 524291 bytes.

2.3438462285897527e-40 - 5.369469272970201e-41im
1.4794447087321173e-40 - 3.1604176345586124e-41im
1.3241689355042352e-40 - 1.888365739486759e-41im
7.175153395966892e-41 + 2.7183577320781074e-42im
3.46185821700549e-42 + 3.7607464275386915e-41im
4.110943542367504e-41 + 2.0568438929256124e-41im
1.7292728326801232e-43 + 2.1007201011334795e-41im
-2.7222719019885807e-42 + 6.325573399908914e-41im
-6.37885377311198e-42 + 

In [270]:
println("Time taken: ", end_time - start, " seconds")

Time taken: 0.006723880767822266 seconds


In [271]:
using FFTW

start = time()
x = fft(input_array)
ent = time()

for a in x
    println(a)
    println()
end

0.9999999999897371 + 0.0im

131072.24998792697 + 1.5707993227664905im

-0.3333367240723126 - 7.989564555325709e-6im

-0.12500107278255507 - 4.494122164867662e-6im

-0.06666720910194392 - 3.195819330475668e-6im

-0.04166699770753095 - 2.4967312395262473e-6im

-0.028571652684777963 - 2.054454686342523e-6im

-0.020833495493452498 - 1.7477150062777313e-6im

-0.015873138800064315 - 1.5218181393764047e-6im

-0.012500096463059138 - 1.3482376562892354e-6im

-0.010101087847672883 - 1.2105371004381613e-6im

-0.008333397344494661 - 1.098561781410807e-6im

-0.006993060622334447 - 1.0056769538301354e-6im

-0.005952426539396096 - 9.273576574574592e-7im

-0.0051282443578834335 - 8.604124754321616e-7im

-0.004464319826711215 - 8.025255242303357e-7im

-0.003921598567983819 - 7.519571169977332e-7im

-0.003472248710224229 - 7.074059830602361e-7im

-0.003095998829800167 - 6.678566332275208e-7im

-0.0027777989336419365 - 6.32505453459735e-7im

-0.002506284737480516 - 6.007175873125997e-7im

-0.002272744556

Excessive output truncated after 524288 bytes.

1.11420967344841e-9im

-8.642113745886194e-9 - 1.114093658267634e-9im

-8.640532665441649e-9 - 1.1139550080108249e-9im

-8.638847220689317e-9 - 1.1138869411167168e-9im

-8.637242078186907e-9 - 1.1137696697096674e-9im

-8.63560804659057e-9 - 

In [272]:
println("Time taken: ", ent - start, " seconds")

Time taken: 0.004083871841430664 seconds


In [312]:
# # Define the QFT MPO
# function qft_mpo(N::Int, sites)
#     # Initialize MPO with sites
#     W = MPO(sites,"I")  
    
#     cutoff=1e-15
#     # Apply Hadamard and phase gates to build the QFT MPO
#     r_max = N
    
#     for i in 1:N
#         W = apply(op("H",sites[1]),W,cutoff=cutoff)

#         for j in 2:r_max
            
#             P = phase_gate(j,j-1,j)
#             swap = swap_gate(j-1,j)
#             # Update MPO by combining with phase gate tensor
#             W = apply(P,W,cutoff=cutoff)
#             W = apply(swap,W,cutoff=cutoff)

#         end
        
#     r_max = r_max - 1    
#     end
    
#     return W
# end

# # Generate the QFT MPO
# W = qft_mpo(N, s)
# println(maxlinkdim(W))
# start = time()
# # Contract MPO and MPS
# result = contract(W, ψ)

# end_time = time()

In [313]:
## Testing Input format ##

In [314]:

# N1 = 4
# s1 = siteinds("Qubit",N1)


In [315]:
# # Taking the input 

# x = range(0, stop=2π, length=2^N1)
# input_array1 = cos.(x)
    
# #ψ = MPS(input_array,s;cutoff=cutoff,maxdim=maxdim)

# MPS9 = []
# basis_states9 = generate_basis_states(N1)

# for i in 1:(2^N1)
#     push!(MPS9, MPS(s1,string.(basis_states9[i])))
# end


# array1 = input_array1 / norm(input_array1) # Input

# ψ1 = array1[1] *MPS9[1]
# for i in 2:2^N1
    
#     ψ1 += array1[i] * MPS9[i]
# end

# println(maxlinkdim(ψ1))

In [316]:
# # ψ3 = MPS(input_array1,s1)
# cutoff1 = 1E-8
# maxdim1 = 10
# T = ITensor(array1,s1)
# M = MPS(T,s1;cutoff=cutoff1,maxdim=maxdim1)


In [317]:
# # println(ψ1[1]*ψ1[2]*ψ1[3])

# println("---")

# # println(ψ3[1]*ψ3[2]*ψ3[3]*ψ3[4])

# # println(M[1]*M[2]*M[3])

# println(T)
# println(array1)

# array9 = Array(T,s1)

# println(array9)